In [1]:
#r "nuget: Azure.AI.Projects, 1.0.0-beta.2"

Installed Packages Azure.AI.Projects, 1.0.0-beta.2

In [2]:
var connectionString = "Your Azure AI Agent Service Connection String";

In [3]:
#r "nuget: Azure.Identity, 1.13.1"

Installed Packages Azure.Identity, 1.13.1

In [4]:

using System;
using System.Collections.Generic;
using System.Threading.Tasks;

using Azure.Identity;
using Azure.AI.Projects;
using Azure.Core;
using Azure.Core.Pipeline;

In [5]:
internal class CustomHeadersPolicy : HttpPipelineSynchronousPolicy
{
    public override void OnSendingRequest(HttpMessage message)
    {
        message.Request.Headers.Add("x-ms-enable-preview", "true");
    }
}

In [6]:
var clientOptions = new AIProjectClientOptions();

clientOptions.AddPolicy(new CustomHeadersPolicy(), HttpPipelinePosition.PerCall);

In [7]:
var projectClient = new AIProjectClient(connectionString, new DefaultAzureCredential(), clientOptions);

In [8]:
ConnectionResponse bingConnection = await projectClient.GetConnectionsClient().GetConnectionAsync("kinfey-bing-search");
var connectionId = bingConnection.Id;

In [9]:
AgentsClient agentClient = projectClient.GetAgentsClient();

In [10]:
ToolConnectionList connectionList = new ToolConnectionList
{
            ConnectionList = { new ToolConnection(connectionId) }
};
BingGroundingToolDefinition bingGroundingTool = new BingGroundingToolDefinition(connectionList);

In [11]:
Azure.Response<Agent> agentResponse = await agentClient.CreateAgentAsync(
           model: "gpt-4",
           name: "web-search-assistant",
           instructions: @"
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.",
           tools: new List<ToolDefinition> { bingGroundingTool });
Agent agent = agentResponse.Value;

In [12]:
Azure.Response<AgentThread> threadResponse = await agentClient.CreateThreadAsync();
AgentThread thread = threadResponse.Value;

In [13]:
Azure.Response<ThreadMessage> messageResponse = await agentClient.CreateMessageAsync(
            thread.Id,
            MessageRole.User,
            "What's Microsoft?");
ThreadMessage message = messageResponse.Value;

In [14]:
Azure.Response<ThreadRun> runResponse = await agentClient.CreateRunAsync(thread, agent);

In [15]:
do
{
            await Task.Delay(TimeSpan.FromMilliseconds(500));
            runResponse = await agentClient.GetRunAsync(thread.Id, runResponse.Value.Id);
}
while (runResponse.Value.Status == RunStatus.Queued
            || runResponse.Value.Status == RunStatus.InProgress);

In [16]:
Azure.Response<PageableList<ThreadMessage>> afterRunMessagesResponse
            = await agentClient.GetMessagesAsync(thread.Id);
IReadOnlyList<ThreadMessage> messages = afterRunMessagesResponse.Value.Data;

In [17]:
foreach (ThreadMessage threadMessage in messages)
{
            Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
            foreach (MessageContent contentItem in threadMessage.ContentItems)
            {
                if (contentItem is MessageTextContent textItem)
                {
                    Console.Write(textItem.Text);
                }
                else if (contentItem is MessageImageFileContent imageFileItem)
                {
                    Console.Write($"<image from ID: {imageFileItem.FileId}");
                }
                Console.WriteLine();
            }
}

2024-12-31 00:03:02 -  assistant: Microsoft Corporation is a multinational technology company headquartered in Redmond, Washington. It was founded by Bill Gates and Paul Allen on April 4, 1975. Microsoft is known for its broad range of software, hardware, and services. Its most prominent products include the Windows operating systems, the Microsoft Office suite, and the Internet Explorer and Edge web browsers. The company also produces hardware for consumer electronics, such as the Xbox video game consoles and the Surface lineup of touchscreen personal computers. Additionally, Microsoft offers cloud-based services like Azure and Office 365, which have become significant parts of its business operations. Microsoft is considered one of the largest and most influential companies in the personal computer industry and technology sector at large.
2024-12-31 00:03:00 -       user: What's Microsoft?
